$$\textbf{PLN. Tarea 2: Minería de texto básica}$$
$$\textit{Y. Sarahi García Gozález}$$

In [ ]:
import re
import os
from keras.preprocessing.text import Tokenizer
from collections import Counter
import matplotlib.pylab as plt
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
#clasificacion
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_fscore_support,roc_auc_score

: 

In [4]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)
print("NLTK version", nltk.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.5
Numpy version 1.23.5
NLTK version 3.8.1


<font size=4.5 color='lightblue'>

$\textit{Ejercicio 2. Bolsas de Palabras, Bigramas y Emociones}$

In [26]:
def get_text_from_file(path_corpus,path_truth):

    tr_txt=[]
    tr_labels=[]

    with open(path_corpus, "r") as f_corpus,open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_txt += [tweet]
        for label in f_truth:
            tr_labels += [label]   
             
    return tr_txt, tr_labels


def create_corpus_from_text(text,tokenizer):
    corpus_palabras=[]
    for documento in text:
        corpus_palabras+=tokenizer.tokenize(documento)

    return corpus_palabras


def create_dic_freq(corpus,n):
    fdist = nltk.FreqDist(corpus)
    aux=[(fdist[key],key) for key in fdist]
    aux.sort()
    aux.reverse()
    aux=aux[:n]

    return aux

def create_dic_ranking(dic_freq):
    dict_indices=dict()
    cont = 0
    for weight, word in dic_freq:
        dict_indices[word]= cont
        cont+= 1

    return dict_indices



In [15]:
tr_txt,tr_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/lenguaje/practicas/03_practica/mex20_train.txt","/Users/ely/Documents/Maestria/segundo_semestre/lenguaje/practicas/03_practica/mex20_train_labels.txt")

In [22]:
tokenizer=TweetTokenizer()
corpus_palabras=create_corpus_from_text(tr_txt,tokenizer)

In [29]:
dict_freq=create_dic_freq(corpus_palabras,5000)

In [30]:
dict_indices=create_dic_ranking(dict_freq)

In [52]:
dict_indices

{',': 0,
 'de': 1,
 'que': 2,
 '.': 3,
 'la': 4,
 'a': 5,
 'y': 6,
 '!': 7,
 'no': 8,
 '@USUARIO': 9,
 'me': 10,
 'el': 11,
 'en': 12,
 'se': 13,
 'es': 14,
 'con': 15,
 '?': 16,
 'verga': 17,
 'los': 18,
 'madre': 19,
 'por': 20,
 'las': 21,
 '"': 22,
 'un': 23,
 'te': 24,
 'mi': 25,
 'lo': 26,
 'putas': 27,
 'una': 28,
 '...': 29,
 'putos': 30,
 'para': 31,
 '😂': 32,
 'si': 33,
 'ya': 34,
 'como': 35,
 'su': 36,
 'pero': 37,
 'tu': 38,
 'loca': 39,
 'le': 40,
 'más': 41,
 'No': 42,
 'del': 43,
 'gorda': 44,
 'al': 45,
 'bien': 46,
 'A': 47,
 '¿': 48,
 'Y': 49,
 'son': 50,
 'Me': 51,
 'o': 52,
 'feas': 53,
 'cuando': 54,
 'Que': 55,
 ':': 56,
 'yo': 57,
 'les': 58,
 'porque': 59,
 'ni': 60,
 'está': 61,
 'ser': 62,
 'estoy': 63,
 'sus': 64,
 'todos': 65,
 'esta': 66,
 'puta': 67,
 'Ya': 68,
 'todo': 69,
 'pinche': 70,
 'puto': 71,
 'tan': 72,
 'Si': 73,
 'La': 74,
 'qué': 75,
 '…': 76,
 'eso': 77,
 'muy': 78,
 'soy': 79,
 'hasta': 80,
 'así': 81,
 '¡': 82,
 '<URL>': 83,
 'mamar': 84,


<font size=2.5 color='lightblue'>

1. Evalué BoW con pesado binario

In [31]:
#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_binary_bow(tr_txt,dic_freq,dic_indices):
    BoW=np.zeros((len(tr_txt),len(dic_freq)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS

    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_indices: #si la palabra está en el corte final
                #AGREGAMOS un uno en el lugar correspondiente
                BoW[cont_documento,dic_indices[word]] = 1 

        cont_documento+=1

    return BoW

In [33]:
binary_bow=build_binary_bow(tr_txt,dict_freq,dict_indices)

<font size=2.5 color='lightblue'>

2. Evalué BoW con pesado de frecuencia

In [35]:
#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_freq_bow(tr_txt,V,dic_inices):
    BoW=np.zeros((len(tr_txt),len(V)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS

    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_inices: #si la palabra está en el corte final
                #SUMAMOS un uno en el lugar correspondiente
                BoW[cont_documento,dic_inices[word]] += 1 

        cont_documento+=1

    return BoW

<font size=2.5 color='lightblue'>

3. Evalué BoW con pesado tfidf

In [121]:
def idf(tr_txt, dic_indices):
    # Crear un diccionario para almacenar el número de líneas en las que aparece cada palabra
    num_lineas_palabra = {palabra: 0 for palabra in dic_indices.keys()}
    # Iterar sobre cada documento en tr_txt
    for documento in tr_txt:
        # Hacer freqDist de cada documento
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(documento))
        unicas=set(fdist_doc)
        # Para cada palabra en el diccionario del documento
        for palabra in unicas:
            # Si la palabra está en el diccionario de frecuencias
            if palabra in dic_indices:
                # Incrementar el contador de líneas para esa palabra
                num_lineas_palabra[palabra] += 1
    return num_lineas_palabra

In [122]:
idf_1=idf(tr_txt, dict_indices)

In [129]:
#ejemplo de juguete para revisar que las funciones hacer lo que deberías:
texto_prueba=['hola, mama y papa, hola\n',
              'hola bebe\n',
              'hola ninia\n',
              'no, mama\n',
              'mama, mama, mama,papa']
corpus1=create_corpus_from_text(texto_prueba,tokenizer)
feq1=create_dic_freq(corpus1,15)
freq2=create_dic_ranking(feq1)
idf_2=idf(texto_prueba,freq2)


In [131]:
#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_tfidf_bow(tr_txt,V,dic_inices,df):
    BoW=np.zeros((len(tr_txt),len(V)),dtype=int) #construimos bolsta de palabras en ceros
    
    cont_documento=0 #indice que recorre las FILAS

    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento        
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_inices: #si la palabra está en el corte final
                
                BoW[cont_documento,dic_inices[word]] += 1 
                
            BoW[cont_documento,dic_inices[word]]=BoW[cont_documento,dic_inices[word]]*np.log(len(tr_txt)/df[word])

        cont_documento+=1
    return BoW

array([0, 0])

<font size=2.5 color='lightblue'>

4. Evalué BoW con pesado binario normalizado l2

<font size=2.5 color='lightblue'>

5. Evalué BoW con pesado frecuencia normalizado l2

<font size=2.5 color='lightblue'>

6. Evalué BoW con pesado tfidf normalizado l2

<font size=2.5 color='lightblue'>

7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

<font size=2.5 color='lightblue'>

8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos (e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.


<font size=2.5 color='lightblue'>

9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado "EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones (BoE).


<font size=2.5 color='lightblue'>

10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen con los tres pesados, normalize cada uno si lo cree conveniente.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 3. Recurso Línguistico de Emociones Mexicano}$

<font size=2.5 color='lightblue'>

11. Utiliceelrecursoléxicollamado"SpanishEmotionLexicon(SEL)"delDr.GrigoriSidorov, profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional (http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emo- ción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla compara- tiva a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente.


<font size=2.5 color='lightblue'>

12. En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el "Probability Factor of Affective use". No más de 5 renglones.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 4. ¿Podemos mejorar con Bigramas?}$


<font size=2.5 color='lightblue'>

13. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteri- ores con otra BoW construida a partir de los 1000 bigramas más frecuentes.


<font size=2.5 color='lightblue'>

14. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi- gramas; usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.


<font size=2.5 color='lightblue'>

15. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea breve: todo en NO más de dos párrafos.